In [1]:
# ********************************************************************************
# IMPORTANT: BEFORE RUNNING THIS CELL, PERFORM A "FACTORY RESET RUNTIME" (Colab)
# OR THE EQUIVALENT DEEPEST RESTART IN YOUR ENVIRONMENT (e.g., Kaggle Session Restart).
# THEN, RUN THIS CELL AS THE VERY FIRST CODE IN YOUR NOTEBOOK.
# ********************************************************************************
!pip install -U scikit-learn==1.3.2 imbalanced-learn==0.12.3
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from collections import Counter
import ast
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

import gc


# Now, the imports should work if the environment is truly clean
try:
    from imblearn.over_sampling import SMOTE
    print("\nSuccessfully imported SMOTE.")
except ImportError as e:
    print(f"\nCRITICAL ERROR: Failed to import SMOTE even after aggressive reinstallation: {e}")
    print("This indicates a severe, persistent environment issue.")
    print("Please double-check that you performed a 'Factory reset runtime' (Colab) or equivalent.")
    exit()

# 1. Inspect and Load GeoJSON Files (Modified for Zero Imputation)
data_dir = "/kaggle/input/mar-oct"  # Replace with your folder path
all_features = []
all_labels = []
invalid_samples = []
invalid_bands = Counter()
species_counts = Counter()

# Updated bands list to include all relevant bands
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'NDVI', 'EVI', 'SAVI', 'NDWI', 'DEM']
months = ['', '_1', '_2', '_3', '_4', '_5', '_6', '_7']
band_columns = [band + month for month in months for band in bands]

# Inspect first file
first_file = os.path.join(data_dir, os.listdir(data_dir)[0]) if os.listdir(data_dir) else None
if first_file and first_file.endswith(".geojson"):
    gdf = gpd.read_file(first_file)
    print("Inspecting first 2 rows of first GeoJSON file:")
    for idx in range(min(2, len(gdf))):
        print(f"\nRow {idx}:")
        for band in ['B1', 'B2', 'B11', 'NDVI', 'DEM', 'B2_1', 'NDVI_7']:
            if band in gdf.columns:
                data = gdf[band].iloc[idx]
                try:
                    parsed_data = ast.literal_eval(data) if isinstance(data, str) else data
                    array = np.array(parsed_data, dtype=np.float32)
                    print(f"  Band {band}: shape={array.shape}, first few values={array.flatten()[:5]}")
                except (ValueError, SyntaxError, TypeError) as e:
                    print(f"  Band {band}: Error parsing/converting: {e}")
            else:
                print(f"  Band {band}: Not found in GeoJSON file")

# Load all GeoJSON files
total_samples_attempted = 0
for file in os.listdir(data_dir):
    if file.endswith(".geojson"):
        try:
            gdf = gpd.read_file(os.path.join(data_dir, file))
            print(f"Processing file: {file}, Rows: {len(gdf)}")
            total_samples_attempted += len(gdf)
            for idx, row in gdf.iterrows():
                try:
                    patch = []
                    for col in band_columns:
                        if col not in gdf.columns:
                            invalid_bands[col] += 1
                            array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                            patch.append(array)
                            continue
                        data = row[col]
                        if data is None or (isinstance(data, str) and data.lower() == 'none'):
                            invalid_bands[col] += 1
                            array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                            patch.append(array)
                            continue
                        try:
                            parsed_data = ast.literal_eval(data) if isinstance(data, str) else data
                            array = np.array(parsed_data, dtype=np.float32).reshape(5, 5)
                        except (ValueError, SyntaxError, TypeError) as e:
                            invalid_bands[col] += 1
                            array = np.zeros((5, 5), dtype=np.float32)  # Impute for parsing errors
                            patch.append(array)
                            continue
                        patch.append(array)
                    patch = np.stack(patch, axis=-1)
                    if patch.shape != (5, 5, 136):  # Expected shape: 17 bands * 8 months
                        raise ValueError(f"Unexpected patch shape: {patch.shape}")
                    all_features.append(patch)
                    all_labels.append(row['l3_species'])
                    species_counts[row['l3_species']] += 1
                except (ValueError, SyntaxError, TypeError) as e:
                    invalid_samples.append((file, idx, str(e)))
                    continue
        except Exception as e:
            print(f"Failed to process file {file}: {e}")
            continue

# Log invalid samples and bands
print(f"\nTotal samples attempted: {total_samples_attempted}")
print(f"Valid samples processed: {len(all_features)}")
if invalid_samples:
    print(f"\nSkipped {len(invalid_samples)} invalid samples:")
    for file, idx, error in invalid_samples:
        print(f"File: {file}, Row: {idx}, Error: {error}")
if invalid_bands:
    print("\nBands with None or missing values:")
    for band, count in invalid_bands.most_common():
        print(f"  {band}: {count} times")
print("\nValid samples per species:")
for species, count in species_counts.most_common():
    print(f"  {species}: {count}")

# Convert to NumPy arrays
if not all_features:
    print("\nError: No valid samples loaded. Using Random Forest with dummy data.")
    X_dummy = np.random.rand(100, 5*5*136)  # Updated for 136 channels
    y_dummy = np.random.randint(0, 5, 100)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_dummy, y_dummy)
    print("Random Forest dummy accuracy:", rf.score(X_dummy, y_dummy))
    print("Please re-export data with updated GEE code.")
    exit()

X = np.array(all_features, dtype=np.float32)  # Shape: (N, 5, 5, 136)
y = np.array(all_labels)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


2025-07-18 21:47:23.953379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752875244.160902      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752875244.216998      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Successfully imported SMOTE.
Inspecting first 2 rows of first GeoJSON file:

Row 0:
  Band B1: shape=(5, 5), first few values=[0.1118 0.1158 0.1158 0.1158 0.1158]
  Band B2: shape=(5, 5), first few values=[0.11225 0.11335 0.11335 0.11955 0.11955]
  Band B11: shape=(5, 5), first few values=[0.17015 0.1891  0.1891  0.1891  0.1891 ]
  Band NDVI: shape=(5, 5), first few values=[0.27416557 0.2273243  0.2273243  0.28395182 0.28395182]
  Band DEM: shape=(5, 5), first few values=[88. 88. 88. 88. 88.]
  Band B2_1: shape=(), first few values=[nan]
  Band NDVI_7: shape=(), first few values=[nan]

Row 1:
  Band B1: shape=(5, 5), first few values=[0.125 0.124 0.124 0.124 0.124]
  Band B2: shape=(5, 5), first few values=[0.1239 0.1176 0.1176 0.1152 0.117 ]
  Band B11: shape=(5, 5), first few values=[0.1976 0.1797 0.1797 0.1797 0.1779]
  Band NDVI: shape=(5, 5), first few values=[0.40535372 0.36756483 0.36756483 0.36180538 0.34991294]
  Band DEM: shape=(5, 5), first few values=[94. 94. 93. 93. 93.]


In [2]:
import json
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, recall_score, f1_score, confusion_matrix, precision_score
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# 1. Encode class labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 2. Flatten features for initial split
X_flat = X.reshape(X.shape[0], -1)

# 3. Initial train/temp split
X_train_raw, X_temp, y_train_raw, y_temp = train_test_split(
    X_flat, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42
)

# 4. Apply SMOTE only to training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_raw, y_train_raw)

# 5. Reshape back to original dimensions
X_train_resampled = X_train_resampled.reshape(-1, 5, 5, 136)
X_temp = X_temp.reshape(-1, 5, 5, 136)

# 6. Convert to one-hot encoding
y_train_onehot = tf.keras.utils.to_categorical(y_train_resampled)
y_temp_onehot = tf.keras.utils.to_categorical(y_temp)
num_classes = len(label_encoder.classes_)

# 7. Split temp into val and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp_onehot, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"\nData shape after SMOTE: {X_train_resampled.shape}, Number of classes: {num_classes}")
print(f"Data range: min={X_train_resampled.min():.4f}, max={X_train_resampled.max():.4f}")

# 8. Compute class weights from the resampled training data
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = dict(enumerate(class_weights))

# 9. Define ResNet-18 Model
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    """A ResNet residual block with shortcut connection."""
    shortcut = x
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

def build_resnet18(input_shape=(5, 5, 136), num_classes=num_classes):
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution
    x = layers.Conv2D(64, 3, strides=1, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # ResNet blocks (4 stages with 2 residual blocks each)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    
    x = residual_block(x, 128, stride=2, conv_shortcut=True)
    x = residual_block(x, 128)
    
    x = residual_block(x, 256, stride=2, conv_shortcut=True)
    x = residual_block(x, 256)
    
    x = residual_block(x, 512, stride=2, conv_shortcut=True)
    x = residual_block(x, 512)
    
    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model3 = models.Model(inputs, outputs, name='resnet18')
    return model3

model3 = build_resnet18()
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 10. Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
])

# 11. Train Model
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
history = model3.fit(
    data_augmentation(X_train_resampled), y_train_onehot,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('best_model3.keras', save_best_only=True),
        lr_scheduler
    ]
)

# 12. Evaluate Model
test_loss, test_accuracy = model3.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")

# 13. Additional Metrics
y_pred = model3.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
report3 = classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_, output_dict=True)
print("\nClassification Report (Test Set):")
print(json.dumps(report3, indent=4))
print(f"Precision: {precision_score(y_test_classes, y_pred_classes, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test_classes, y_pred_classes, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_test_classes, y_pred_classes, average='weighted'):.4f}")

# 14. Confusion Matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Test Set)')
plt.savefig('confusion3.png')
plt.close()

# 15. Save Outputs
model3.save('model3.keras')  # Better to save as .keras format for TensorFlow models
json.dump(report3, open('report3.json', 'w'), indent=4)
np.save('confusion3.npy', cm)
joblib.dump(label_encoder, 'labelencoder3.pkl')

print("Saved: model3.keras, report3.json, confusion3.npy, confusion3.png, labelencoder3.pkl")


Data shape after SMOTE: (71668, 5, 5, 136), Number of classes: 19
Data range: min=-383.2718, max=742.0000


I0000 00:00:1752875645.875785      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/50


I0000 00:00:1752875675.188963      67 service.cc:148] XLA service 0x61211670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752875675.189732      67 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752875677.467006      67 cuda_dnn.cc:529] Loaded cuDNN version 90300


   8/1120 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.0831 - loss: 3.4725

I0000 00:00:1752875687.833734      67 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1120/1120 ━━━━━━━━━━━━━━━━━━━━ 70s 30ms/step - accuracy: 0.2056 - loss: 2.6242 - val_accuracy: 0.2796 - val_loss: 2.1905 - learning_rate: 1.0000e-04
Epoch 2/50
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.3323 - loss: 2.0596 - val_accuracy: 0.2990 - val_loss: 2.2718 - learning_rate: 1.0000e-04
Epoch 3/50
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - accuracy: 0.3797 - loss: 1.8894 - val_accuracy: 0.3415 - val_loss: 1.9422 - learning_rate: 1.0000e-04
Epoch 4/50
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - accuracy: 0.4262 - loss: 1.7539 - val_accuracy: 0.3871 - val_loss: 1.8262 - learning_rate: 1.0000e-04
Epoch 5/50
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.4644 - loss: 1.6359 - val_accuracy: 0.3639 - val_loss: 2.0545 - learning_rate: 1.0000e-04
Epoch 6/50
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - accuracy: 0.5071 - loss: 1.5086 - val_accuracy: 0.4611 - val_loss: 1.6566 - learning_rate: 1.0000e-04
Epoch 7/50
1120/1120 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/

In [3]:
# 16. Evaluate Model on Final Test Data
import os
import ast
import pandas as pd
import geopandas as gpd
from collections import Counter

test_data_dir = "/kaggle/input/final-test-data"
test_features = []
test_labels = []
test_invalid_samples = []
test_invalid_bands = Counter()
total_samples_attempted = 0  # Track total samples processed

# Check if directory exists and list files
if not os.path.exists(test_data_dir):
    print(f"\nError: Test data directory {test_data_dir} does not exist.")
    exit()
geojson_files = [f for f in os.listdir(test_data_dir) if f.endswith(".geojson")]
print(f"\nFound {len(geojson_files)} GeoJSON files in {test_data_dir}")

# Load all GeoJSON files from test data directory
for file in geojson_files:
    try:
        file_path = os.path.join(test_data_dir, file)
        gdf = gpd.read_file(file_path)
        print(f"Processing file: {file}, Rows: {len(gdf)}")
        total_samples_attempted += len(gdf)  # Count all rows in the file
        for idx, row in gdf.iterrows():
            try:
                patch = []
                for col in band_columns:
                    if col not in gdf.columns:
                        test_invalid_bands[col] += 1
                        array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                        patch.append(array)
                        continue
                    data = row[col]
                    if data is None or (isinstance(data, str) and data.lower() == 'none'):
                        test_invalid_bands[col] += 1
                        array = np.zeros((5, 5), dtype=np.float32)  # Impute with zeros
                        patch.append(array)
                        continue
                    try:
                        parsed_data = ast.literal_eval(data) if isinstance(data, str) else data
                        array = np.array(parsed_data, dtype=np.float32).reshape(5, 5)
                    except (ValueError, SyntaxError, TypeError) as e:
                        test_invalid_bands[col] += 1
                        array = np.zeros((5, 5), dtype=np.float32)  # Impute for parsing errors
                        patch.append(array)
                        continue
                    patch.append(array)
                patch = np.stack(patch, axis=-1)
                if patch.shape != (5, 5, 136):  # Expected shape: 17 bands * 8 months
                    raise ValueError(f"Unexpected patch shape: {patch.shape}")
                test_features.append(patch)
                test_labels.append(row['l3_species'])
            except (ValueError, SyntaxError, TypeError) as e:
                test_invalid_samples.append((file, idx, str(e)))
                continue
    except Exception as e:
        print(f"Failed to process file {file}: {e}")
        continue

# Log invalid samples and bands
print(f"\nTotal samples attempted: {total_samples_attempted}")
print(f"Valid samples processed: {len(test_features)}")
if test_invalid_samples:
    print(f"\nSkipped {len(test_invalid_samples)} invalid test samples:")
    for file, idx, error in test_invalid_samples:
        print(f"File: {file}, Row: {idx}, Error: {error}")
if test_invalid_bands:
    print("\nBands with missing/None/parsing issues in test data:")
    for band, count in test_invalid_bands.most_common():
        print(f"  {band}: {count} times")

# Convert to NumPy arrays
if not test_features:
    print("\nError: No valid test samples loaded. Cannot evaluate model.")
    exit()

X_test_final = np.array(test_features, dtype=np.float32)  # Shape: (N, 5, 5, 136)
y_test_final = np.array(test_labels)

# Preprocess test data
try:
    y_test_final_encoded = label_encoder.transform(y_test_final)  # Use same LabelEncoder
except ValueError as e:
    print(f"Error in label encoding: {e}")
    unknown_labels = set(y_test_final) - set(label_encoder.classes_)
    print(f"Unknown labels in test data: {unknown_labels}")
    exit()

# Convert to one-hot encoding for neural network
y_test_final_onehot = tf.keras.utils.to_categorical(y_test_final_encoded, num_classes=num_classes)

# Diagnostic: Compare class distributions
print("\nTraining class distribution:")
print(pd.Series(label_encoder.inverse_transform(y_train_resampled)).value_counts())
print("\nTest class distribution:")
print(pd.Series(y_test_final).value_counts())
missing_classes = set(label_encoder.classes_) - set(y_test_final)
print(f"Classes missing in test data: {missing_classes}")

# Evaluate model on test data
y_pred_final = model3.predict(X_test_final)
y_pred_final_classes = np.argmax(y_pred_final, axis=1)
test_accuracy_final = (y_pred_final_classes == y_test_final_encoded).mean()
print(f"\nFinal Test Data Accuracy: {test_accuracy_final:.4f}")

# Get unique labels in test data to avoid mismatch
unique_test_labels = np.unique(y_test_final_encoded)
unique_test_label_names = label_encoder.inverse_transform(unique_test_labels)

# Additional metrics for test data
report_final = classification_report(
    y_test_final_encoded,
    y_pred_final_classes,
    labels=unique_test_labels,
    target_names=unique_test_label_names,
    output_dict=True
)
print("\nClassification Report for Final Test Data:")
print(json.dumps(report_final, indent=4))
print(f"Precision (Final Test): {precision_score(y_test_final_encoded, y_pred_final_classes, average='weighted'):.4f}")
print(f"Recall (Final Test): {recall_score(y_test_final_encoded, y_pred_final_classes, average='weighted'):.4f}")
print(f"F1-Score (Final Test): {f1_score(y_test_final_encoded, y_pred_final_classes, average='weighted'):.4f}")

# Confusion matrix for test data
cm_final = confusion_matrix(y_test_final_encoded, y_pred_final_classes, labels=unique_test_labels)
plt.figure(figsize=(10, 8), dpi=100)
sns.heatmap(cm_final, annot=True, fmt='d', xticklabels=unique_test_label_names, yticklabels=unique_test_label_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Final Test Data')
plt.savefig('confusion_final.png', dpi=100)
plt.close()

# Total number of test points
print(f"\nTotal Number of Test Points: {len(y_test_final)}")

# Save outputs for test data
json.dump(report_final, open('report_final.json', 'w'), indent=4)
np.save('confusion_final.npy', cm_final)
print("Saved: report_final.json, confusion_final.npy, confusion_final.png")


Found 19 GeoJSON files in /kaggle/input/final-test-data
Processing file: needleleaf_douglas fir_douglas firmar-oct-2022.geojson, Rows: 506
Processing file: broadleaf_short-lived deciduous_aldermar-oct-2022.geojson, Rows: 420
Processing file: broadleaf_beech_european beechmar-oct-2022.geojson, Rows: 1703
Processing file: needleleaf_larch_japanese larchmar-oct-2022.geojson, Rows: 135
Processing file: broadleaf_short-lived deciduous_poplarmar-oct-2022.geojson, Rows: 77
Processing file: needleleaf_pine_scots pinemar-oct-2022.geojson, Rows: 1202
Processing file: broadleaf_oak_english oakmar-oct-2022.geojson, Rows: 645
Processing file: needleleaf_larch_european larchmar-oct-2022.geojson, Rows: 221
Processing file: broadleaf_long-lived deciduous_sycamore maplemar-oct-2022.geojson, Rows: 725
Processing file: broadleaf_long-lived deciduous_european ashmar-oct-2022.geojson, Rows: 432
Processing file: broadleaf_short-lived deciduous_birchmar-oct-2022.geojson, Rows: 353
Processing file: broadleaf